In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

In [2]:
df = pd.read_csv('county_data.csv')
df.head()

,year,state_po,county_name,county_fips,totalvotes,dvotes,rvotes,statevotes,vote_percentage,r_margin_perc
0,2000,AL,AUTAUGA,1001.0,17208,4942,11993,1672551,0.010288,0.004216
1,2000,AL,BALDWIN,1003.0,56480,13997,40872,1672551,0.033769,0.016068
2,2000,AL,BARBOUR,1005.0,10395,5188,5096,1672551,0.006215,-0.000055
3,2000,AL,BIBB,1007.0,7101,2710,4273,1672551,0.004246,0.000935
4,2000,AL,BLOUNT,1009.0,17973,4977,12667,1672551,0.010746,0.004598


In [3]:
with open('counties-albers-10m.json', 'r') as file:
    topo = json.load(file)

In [4]:
states = df["state_po"].unique().tolist()

In [5]:
state_counties = []
for state in states:
    counties = df[df["state_po"] == state]["county_name"].unique()
    state_counties.append(counties)

Use Andrew Carlson Maps, not included in repo

In [6]:
for idx, state in enumerate(tqdm(states)):
    county_names = state_counties[idx]
    with open(f'maps/jsoncounties-{state}.min.js') as file:
        state_json = json.load(file)
    state_df = df[df["state_po"] == state]
    counties = state_json["features"]["counties"]
    for county_idx, county_name in enumerate(county_names):
        county_name = county_name.lower()
        for county in counties:
            if county["name"].lower() == county_name:
                county["data"] = []
                county_df = state_df[state_df["county_name"] == county_names[county_idx]][["year", "totalvotes", "dvotes", "rvotes", "statevotes", "vote_percentage", "r_margin_perc"]]
                for row in county_df.itertuples():
                    datapoint = {
                        "year":row.year,
                        "totalvotes":row.totalvotes,
                        "dvotes":row.dvotes,
                        "rvotes":row.rvotes,
                        "vote_perc":row.vote_percentage,
                        "r_marg":row.r_margin_perc
                    }
                    county["data"].append(datapoint)
                break
            
    with open(f"output/{state}.json", 'w', encoding='utf-8') as file:
        json.dump(state_json, file, ensure_ascii=False, indent=4)

100%|██████████| 51/51 [00:03<00:00, 13.16it/s]
